In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import json


In [3]:
DATA_PATH = "./data/aligned_data/"

In [4]:
with open("./data/new_aligned_tickers_2128.json") as jsf:
    new_tickers = json.load(jsf)

In [8]:
prices = pd.read_csv(f"{DATA_PATH}/prices.csv", parse_dates=True, index_col=0)
returns = pd.read_csv(f"{DATA_PATH}/returns.csv", parse_dates=True, index_col=0)
volume = pd.read_csv(f"{DATA_PATH}/volume.csv", parse_dates=True, index_col=0)
ychopmdf = pd.read_csv(f"{DATA_PATH}/ychopmdf.csv", parse_dates=True, index_col=0)
asset_turnover = pd.read_csv(f"{DATA_PATH}/asset_turnover.csv", parse_dates=True, index_col=0)